# n-gram 언어모델
단어 문장 대신 아이템 넣어서 평가

In [2]:
#코드 참조:
#데이터 사이언스 스쿨: 확률론적 언어 모형(https://datascienceschool.net/view-notebook/a0c848e1e2d343d685e6077c35c4203b/)
from nltk import bigrams, word_tokenize
from nltk.util import ngrams
import nltk
nltk.download("book", quiet=True)
from nltk.book import *

import pandas as pd
import numpy as np
from nltk import ConditionalFreqDist

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [3]:
movies = pd.read_csv('ml-25m/movies.csv')
ratings = pd.read_csv('ml-25m/ratings.csv')

In [4]:
ratings['liked'] = np.where(ratings['rating']>=4, 1, 0)
ratings['movieId'] = ratings['movieId'].astype('str')
gp_user_like = ratings.groupby(['liked', 'userId'])

In [5]:
# 유저 n이 좋아한 영화 => positive example
# 유저 n이 싫어하는 영화 별로 그룹, 좋아하는 영화 별로 그룹핑
splitted_movies = [gp_user_like.get_group(gp)['movieId'].tolist() for gp in gp_user_like.groups]

In [6]:
for movie_list in splitted_movies:
    random.shuffle(movie_list)

NameError: name 'random' is not defined

In [ ]:
len(splitted_movies)

## 아이템셋 토큰화
- window 사이즈 2인 n-gram 모형
- SS: 문장의 처음
- SE: 문장의 끝

In [160]:
sentences = []
for tokens in splitted_movies:
    bigram = ngrams(tokens, 2, pad_left=True, pad_right=True, left_pad_symbol="SS", right_pad_symbol="SE")
    sentences += [t for t in bigram]

In [54]:
print(sentences[:150])

[('SS', '306'), ('306', '899'), ('899', '1175'), ('1175', '1217'), ('1217', '1260'), ('1260', '2011'), ('2011', '2012'), ('2012', '2068'), ('2068', '2161'), ('2161', '4308'), ('4308', '4422'), ('4422', '5269'), ('5269', '5684'), ('5684', '5912'), ('5912', '6539'), ('6539', '6954'), ('6954', '7318'), ('7318', '7323'), ('7323', '7327'), ('7327', '7820'), ('7820', '7937'), ('7937', '7938'), ('7938', '7939'), ('7939', '8014'), ('8014', '8405'), ('8405', '8685'), ('8685', '8729'), ('8729', '8873'), ('8873', '27193'), ('27193', '27721'), ('27721', '31956'), ('31956', 'SE'), ('SS', '1'), ('1', '62'), ('62', '261'), ('261', '266'), ('266', '380'), ('380', '480'), ('480', '524'), ('524', '553'), ('553', '588'), ('588', '653'), ('653', '858'), ('858', '1035'), ('1035', '1080'), ('1080', '1201'), ('1201', '1271'), ('1271', '1299'), ('1299', '1302'), ('1302', '1431'), ('1431', '1465'), ('1465', '1485'), ('1485', '1527'), ('1527', '1587'), ('1587', '1722'), ('1722', '1923'), ('1923', '1957'), ('195

In [132]:
movieId_to_name = pd.Series(movies.title.values, index = movies.movieId.values).to_dict()
name_to_movieId = pd.Series(movies.movieId.values, index = movies.title).to_dict()

In [161]:
cfd = ConditionalFreqDist(sentences)

In [162]:
from nltk.probability import ConditionalProbDist, MLEProbDist
cpd = ConditionalProbDist(cfd, MLEProbDist)

In [73]:
cpd[100]

<MLEProbDist based on 0 samples>

In [60]:
def sentence_score(s):
    p = 0.0
    for i in range(len(s) - 1):
        c = s[i]
        w = s[i + 1]
        p += np.log(cpd[c].prob(w) + np.finfo(float).eps)
    return np.exp(p)

In [149]:
def generate_sentence(seed=None, start_word="SS"):
    if seed is not None:
        import random
        random.seed(seed)
        
    c = start_word
    sentence = []
    sentence.append(c)
    
    while True:
        if c not in cpd:
            break
        w = cpd[c].generate()
        if w == "SE":
            break
        else:
            w2 = w
        sentence.append(w2)
        c = w
    
    moviename = []
    for i in sentence:
        mname = movieId_to_name[int(i)]
        moviename.append(mname)
    
    return sentence, moviename

In [177]:
sentence, movie = generate_sentence(start_word="3")

In [178]:
print(sentence)
print(movie)

['3', '3120', '8958', '7143', '106782', '4816', '6936', '2791', '1032', '160', '435', '2194', '3623', '89745', '1198', '1', '2716', '4814', '1641', '2407', '7781', '4735', '5384', '30749', '4848', '58103', '3986', '6874', '500', '1097', '50', '173343', '553', '231', '2716', '32587', '6863', '40815', '7669', '2427', '3006', '296', '3538', '5225', '63082', '6874', '3996', '2011', '1247', '1721', '3176', '4776', '1391', '7206', '4966', '1587', '8798', '1266', '1250', '529', '1302', '1394', '1240', '714']
['Grumpier Old Men (1995)', 'Distinguished Gentleman, The (1992)', 'Ray (2004)', 'Last Samurai, The (2003)', 'Wolf of Wall Street, The (2013)', 'Zoolander (2001)', 'Elf (2003)', 'Airplane! (1980)', 'Alice in Wonderland (1951)', 'Congo (1995)', 'Coneheads (1993)', 'Untouchables, The (1987)', 'Mission: Impossible II (2000)', 'Avengers, The (2012)', 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)', 'Toy Story (1995)', 'Ghostbusters (a.k.a. Ghost Busters) (1984

## 개선해야할 점
- 조건부 확률... 순서는 상관이 없기 때문에 이 점 개선
- 출발 아이템을 frequent한 것으로 바꿔주기

In [201]:
def find_usr_with_id(mid):
    usr = []
    for i in splitted_movies:
        if mid in i:
            usr.append(i)
    
    # 해당 mid를 본 유저와 생성한 sentence를 비교
    compare = [] # 생성한 sentence와 비교해 일치율 구하기
    for u in usr:
        c = list(set(u).intersection(sentence))
        prob = (len(c) / len(sentence)) * 100
        compare.append(prob)
    
    return usr, compare

In [202]:
usr, compare = find_usr_with_id("3")

In [198]:
compare.index(max(compare))

2840

In [ ]:
compare

4776

In [ ]:
# support 실제 얼마나 나오는지
# 출발도 frequent한 아이템을 가지고 real support 구하기
# association rule로 구하는 것 보다 이게 리즈너블 한가?